#### Patrick Pfenning

Fibromyalgia Final Project:
---

>**OBJECTIVE:** Use the [Fibromyalgia Abstracts](../data/fibro_abstracts.csv) dataset to generate titles from abstract content. Titles behave as a summary of sorts, so title will serve as testing outputs.

>**DELIVERABLES:** [Video Walkthroughs](../recordings)


## Part I:  _Data Cleanup_

### 1. Index Verification

> After loading [Fibromyalgia Abstracts](../data/fibro_abstracts.csv) to a dataframe, it is wise to find a primary key to use as our index. A primary key is a unique set of values for which all data can be mapped. This makes joining other dataframes with a map to the index easy.

>**Key Choice:** _PMID_

>**Justification:** The below shows that the id is unique. Given that it is an integer key, it is easy to reference as the reader.

In [2]:
import pandas as pd

# download set
fibro = pd.read_csv('../data/fibro_abstracts.csv')
# is id truly unique?
if fibro.PMID.is_unique:
    fibro.set_index('PMID', inplace=True)
fibro.head()

,titles,pub_date,source,abstract
PMID,,,,
32314938,"Fibromyalgia, Sjogren's & Depression: Linked?",2020 Apr 21,Postgrad Med. 2020 Apr 21. doi: 10.1080/003254...,Health care has become increasingly fragmented...
32308473,Electrodiagnostic Abnormalities Associated wit...,2020,J Pain Res. 2020 Apr 9;13:737-744. doi: 10.214...,Purpose: Increasing evidence suggests that fib...
32307906,High levels of cathepsin S and cystatin C in p...,2020 Apr 19,Int J Rheum Dis. 2020 Apr 19. doi: 10.1111/175...,OBJECTIVES: Although the etiopathogenesis of f...
32305723,Prevalence and overlap of somatic symptom diso...,2020 Apr 11,J Psychosom Res. 2020 Apr 11;133:110111. doi: ...,OBJECTIVE: To study the prevalence and clinica...
32299495,Psychometric properties of Turkish version of ...,2020 Apr 16,Adv Rheumatol. 2020 Apr 16;60(1):22. doi: 10.1...,BACKGROUND: Fibromyalgia syndrome (FMS) has ad...


### 2. Extract date from pub_date --> (yr, mo, date)

>**_Approach:_**
>>1. Extract from source
>>    - pros: seems more "precise"
>>    - cons: more text to parse (journal, author, doi, ...)
>>2. Extract from pub_date
>>    - pros: already an attempted date with inteded format of "%Y %b %d"
>>    - cons: will need to extrapolate to get missing data
    

>**_Choice:_**
>>__PMID__ seems to be presorted in decending order. This inherently maps to time of entry, thus an __ffill__ >extrapolation will give us a logical approximation for the date unit.

In [3]:
from datetime import datetime as dt
import regex as re
from itertools import cycle, islice, zip_longest

# expect YYYY Mon d
dates = fibro['pub_date'].str.split(expand=True).iloc[:, :3]
dates.columns = ['yr', 'mo', 'day']
dates = dates.fillna(method='ffill').set_index(dates.index)
dates.mo = dates.mo.str.slice(0, 3)
### Will join to finished product 
dates.head()

,yr,mo,day
PMID,,,
32314938,2020,Apr,21
32308473,2020,Apr,21
32307906,2020,Apr,19
32305723,2020,Apr,11
32299495,2020,Apr,16


### 3. Get Publication and DOI from Source

>The __source__ column looks to be a conjunction of __(Publication Title) (Date) (DOI).__ Because we already found date, let us extract to others.

In [4]:
source = fibro.source.str.split(r'(\s\d{4}\s)|(doi:)', expand=True)
print(source.head())
source = source[[0, 6]]
source.columns = ['publication', 'DOI']
source.head() # DOI is okay to be left blank as it id a digital key and may not have existed at time of publication

                         0       1     2                    3     4     5  \
PMID                                                                        
32314938     Postgrad Med.   2020   None             Apr 21.   None  doi:   
32308473       J Pain Res.   2020   None   Apr 9;13:737-744.   None  doi:   
32307906  Int J Rheum Dis.   2020   None             Apr 19.   None  doi:   
32305723  J Psychosom Res.   2020   None  Apr 11;133:110111.   None  doi:   
32299495    Adv Rheumatol.   2020   None    Apr 16;60(1):22.   None  doi:   

                                                6     7     8     9  
PMID                                                                 
32314938           10.1080/00325481.2020.1758426.  None  None  None  
32308473   10.2147/JPR.S234475. eCollection 2020.  None  None  None  
32307906                 10.1111/1756-185X.13840.  None  None  None  
32305723        10.1016/j.jpsychores.2020.110111.  None  None  None  
32299495               10.1186/s42358-02

,publication,DOI
PMID,,
32314938,Postgrad Med.,10.1080/00325481.2020.1758426.
32308473,J Pain Res.,10.2147/JPR.S234475. eCollection 2020.
32307906,Int J Rheum Dis.,10.1111/1756-185X.13840.
32305723,J Psychosom Res.,10.1016/j.jpsychores.2020.110111.
32299495,Adv Rheumatol.,10.1186/s42358-020-0123-3.


### 4. Clean Corpus

> Remove bad characters from title/abstract:
> - punctuation
> - non-ascii

In [5]:
clean_text = fibro[['titles', 'abstract']].copy()
clean_text.columns = ['title', 'abstract']  # don't love the plural column name
print(clean_text.head())
for col in clean_text:
    clean_text[col] = clean_text[col].str.encode('ascii', 'ignore') \
                                     .str.decode('ascii') \
                                     .replace(r'(?:[^\w\s]|_)+', '', regex=True) \
                                     .str.strip()
clean_text.head()

                                                      title  \
PMID                                                          
32314938      Fibromyalgia, Sjogren's & Depression: Linked?   
32308473  Electrodiagnostic Abnormalities Associated wit...   
32307906  High levels of cathepsin S and cystatin C in p...   
32305723  Prevalence and overlap of somatic symptom diso...   
32299495  Psychometric properties of Turkish version of ...   

                                                   abstract  
PMID                                                         
32314938  Health care has become increasingly fragmented...  
32308473  Purpose: Increasing evidence suggests that fib...  
32307906  OBJECTIVES: Although the etiopathogenesis of f...  
32305723  OBJECTIVE: To study the prevalence and clinica...  
32299495  BACKGROUND: Fibromyalgia syndrome (FMS) has ad...  


,title,abstract
PMID,,
32314938,Fibromyalgia Sjogrens Depression Linked,Health care has become increasingly fragmented...
32308473,Electrodiagnostic Abnormalities Associated wit...,Purpose Increasing evidence suggests that fibr...
32307906,High levels of cathepsin S and cystatin C in p...,OBJECTIVES Although the etiopathogenesis of fi...
32305723,Prevalence and overlap of somatic symptom diso...,OBJECTIVE To study the prevalence and clinical...
32299495,Psychometric properties of Turkish version of ...,BACKGROUND Fibromyalgia syndrome FMS has adver...


### 5. Join Cleaned

>Join the above datasets to a finishes product `fibro_cleaned`

In [6]:
# old
fibro.head()

,titles,pub_date,source,abstract
PMID,,,,
32314938,"Fibromyalgia, Sjogren's & Depression: Linked?",2020 Apr 21,Postgrad Med. 2020 Apr 21. doi: 10.1080/003254...,Health care has become increasingly fragmented...
32308473,Electrodiagnostic Abnormalities Associated wit...,2020,J Pain Res. 2020 Apr 9;13:737-744. doi: 10.214...,Purpose: Increasing evidence suggests that fib...
32307906,High levels of cathepsin S and cystatin C in p...,2020 Apr 19,Int J Rheum Dis. 2020 Apr 19. doi: 10.1111/175...,OBJECTIVES: Although the etiopathogenesis of f...
32305723,Prevalence and overlap of somatic symptom diso...,2020 Apr 11,J Psychosom Res. 2020 Apr 11;133:110111. doi: ...,OBJECTIVE: To study the prevalence and clinica...
32299495,Psychometric properties of Turkish version of ...,2020 Apr 16,Adv Rheumatol. 2020 Apr 16;60(1):22. doi: 10.1...,BACKGROUND: Fibromyalgia syndrome (FMS) has ad...


In [7]:
# new
fibro_clean = pd.concat([dates, source, clean_text], axis=1)
fibro_clean.head()

,yr,mo,day,publication,DOI,title,abstract
PMID,,,,,,,
32314938,2020,Apr,21,Postgrad Med.,10.1080/00325481.2020.1758426.,Fibromyalgia Sjogrens Depression Linked,Health care has become increasingly fragmented...
32308473,2020,Apr,21,J Pain Res.,10.2147/JPR.S234475. eCollection 2020.,Electrodiagnostic Abnormalities Associated wit...,Purpose Increasing evidence suggests that fibr...
32307906,2020,Apr,19,Int J Rheum Dis.,10.1111/1756-185X.13840.,High levels of cathepsin S and cystatin C in p...,OBJECTIVES Although the etiopathogenesis of fi...
32305723,2020,Apr,11,J Psychosom Res.,10.1016/j.jpsychores.2020.110111.,Prevalence and overlap of somatic symptom diso...,OBJECTIVE To study the prevalence and clinical...
32299495,2020,Apr,16,Adv Rheumatol.,10.1186/s42358-020-0123-3.,Psychometric properties of Turkish version of ...,BACKGROUND Fibromyalgia syndrome FMS has adver...


## Part II: _Model_
---

>Now that our data is in a cleansed and in an easily queryable state, if is time to analyze the important parts of this document: Title and Abstract. Our goal here is to develop a title from the given abstract. 

> **DISCLAIMER:** I have chosen to use `SimpleT5` as a pretrain solution. I followed this [tutorial](https://colab.research.google.com/drive/1JZ8v9L0w0Ai3WbibTeuvYlytn0uHMP6O?usp=sharing#scrollTo=zWE4rl2vhaLZ) to set my model up. This also requires a pytorxh install.

### 1. Train Test Split

> We want to split our data into a train and test set using `sklearn`.

In [8]:
# build dataset

RANDOMS = 42

# example text in and out
print("--- Title ---")
print(fibro_clean["title"].iloc[0])
print("--- Abstract ---")
print(fibro_clean["abstract"].iloc[0])

# t5 set
df = fibro_clean[['abstract', 'title']].copy()
df.columns = ['source_text', 'target_text']

# T5 model expects a task related prefix: since it is a summarization task, we will add a prefix "summarize: "
df['source_text'] = "summarize: " + df['source_text']
df.head()

--- Title ---
Fibromyalgia Sjogrens  Depression Linked
--- Abstract ---
Health care has become increasingly fragmented partly due to advancing medical technology Patients are often managed by various specialty teams when presenting with symptoms that could be manifestations of different diseases Approximately one third of them are referred to specialists at over half for outpatient appointments1 Fatigue pain depression dry mouth headaches and arthralgia are common complaints and frequently require referral to specialist physicians Differential diagnoses include fibromyalgia FM Sjogrens syndrome SS and depression Evaluations involve various subspecialist especially physicians like those practicing pain management rheumatology and psychiatryThresholds for referring vary Patients sometime feel lost in a medical maze Disagreement is frequent between specialties regarding management12 Each discipline has its own diagnostic and treatment protocols and there is little consensus about shared d

,source_text,target_text
PMID,,
32314938,summarize: Health care has become increasingly...,Fibromyalgia Sjogrens Depression Linked
32308473,summarize: Purpose Increasing evidence suggest...,Electrodiagnostic Abnormalities Associated wit...
32307906,summarize: OBJECTIVES Although the etiopathoge...,High levels of cathepsin S and cystatin C in p...
32305723,summarize: OBJECTIVE To study the prevalence a...,Prevalence and overlap of somatic symptom diso...
32299495,summarize: BACKGROUND Fibromyalgia syndrome FM...,Psychometric properties of Turkish version of ...


In [9]:
# train test split

from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2)
train_df.shape, test_df.shape

((4688, 2), (1173, 2))

In [ ]:
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")
model.train(train_df=train_df[:5000],
            eval_df=test_df[:100], 
            source_max_token_len=128, 
            target_max_token_len=50, 
            batch_size=8, max_epochs=3, use_gpu=False)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Missing logger folder: /Users/ppfenning/PycharmProjects/fibro/deliverables/lightning_logs

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/Caskroom/miniconda/base/envs/fibro/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Global seed set to 42
Global seed set to 42


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Global seed set to 42
/usr/local/Caskroom/miniconda/base/envs/fibro/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...

Validating: 0it [00:00, ?it/s]

Global seed set to 42
Global seed set to 42


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Global seed set to 42
Global seed set to 42


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
